In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier
from graphviz import Source   # Install it using: conda install -c conda-forge python-graphviz
from IPython.display import display, SVG

In [4]:
# Load dataset
liver_patient = pd.read_csv("duomenys/Indian_Liver_Patient.csv")
liver_patient.head()

#pervadiname stulpelį dataset į liver_disease
liver_patient.rename(columns={'Dataset': 'Liver_Disease'}, inplace=True)
liver_patient['Gender'] = liver_patient['Gender'].apply(lambda x: 1 if x =='Male' else 0)
#Female=0 and Male = 1
liver_patient['Albumin_and_Globulin_Ratio'].mean()
liver_patient=liver_patient.fillna(0.94)

# Train/test mask
np.random.seed(42)

mask = np.random.random(liver_patient.shape[0]) < 0.7

liver_patient.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Liver_Disease
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
#Apsiskaiciuojame Globulina
liver_patient['Globulin'] = liver_patient['Albumin'] / liver_patient['Albumin_and_Globulin_Ratio']

#Susiskaiciuojam netiesiogini bilirubina (nekonjuguotas) (jis lygus is bendro atemus tiesiogini)
liver_patient['Indirect_Bilirubin'] = liver_patient['Total_Bilirubin'] - liver_patient['Direct_Bilirubin']

In [27]:
 rf= RandomForestClassifier(n_estimators = 100)

In [28]:
rf.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
pred = rf.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))

In [30]:
acc = np.mean(pred == liver_patient[~mask]['Liver_Disease'])
print(f'Accuracy {acc:.02%}')

Accuracy 71.89%


# Confusion matrix

In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
pd.DataFrame(confusion_matrix(liver_patient[~mask]['Liver_Disease'], pred),
             columns=rf.classes_, index=rf.classes_)

,1,2
1,117,17
2,32,19


# Feature importance

In [40]:
pd.DataFrame(rf.feature_importances_,
             index=liver_patient.drop('Liver_Disease', axis=1).columns)

,0
Age,0.094682
Gender,0.011395
Total_Bilirubin,0.069320
Direct_Bilirubin,0.078928
Alkaline_Phosphotase,0.130941
Alamine_Aminotransferase,0.124857
Aspartate_Aminotransferase,0.106531
Total_Protiens,0.078111
Albumin,0.078529
Albumin_and_Globulin_Ratio,0.073570
